# [01_Dancing_Particles](01_Dancing_Particles.ipynb)


**Exercise:**

A much more stable integrator than the `euler` we used so far is the verlocity Verlet:

$$\left\{\begin{array}{l}
x_{n+1} = x_{n} + v_{n} \Delta t + \frac{F_V(x_{n})}{2} \Delta t^2\\
v_{n+1} = v_{n} + \frac{F_V(x_{n)} + F_V(x_{n+1})}{2} \Delta t\\
\end{array}\right. $$

- Program this algorithm, taking care that it supports multi-dimensional positions and velocities as well. In practice we would like to avoid recomputing $F_V(x)$ as much as possible, since this is usually the expensive step of the dynamics. For our purposes there is no need to keep an eye on that for now.
- How does the previous dynamics look like in this example. Does this algorithm conserve energy (phase-space plot)?
- Also look at the Morse potential

In [ ]:
# One way to code the verlet function is:
function verlet(F, Δt, xₙ, vₙ)
    Fₙ = F(xₙ)
    xₙ₊₁ = xₙ + vₙ * Δt + Fₙ/2 * Δt^2
    vₙ₊₁ = vₙ + (Fₙ + F(xₙ₊₁)) / 2 * Δt
    xₙ₊₁, vₙ₊₁
end

### Exercise
Program the total potential function for a matrix $\textbf{x}$. A useful function is `norm` from the `LinearAlgebra` package.

In [ ]:
# One solution:
function Vtot(Vpair, x)
    n_particles = size(x, 2)
    accu = zero(eltype(x))   # Get a zero of the appropriate type
    for i in 1:n_particles, j in i+1:n_particles
        accu += Vpair(norm(x[:, i] .- x[:, j]))
    end
    accu
end

## Open-ended exercise

#### 1. Performance improvements

The most immediate performance improvements are obtained by using views and disabling the bounds checks, for example:

In [ ]:
function Vtot(Vpair, x)
    n_particles = size(x, 2)
    accu = zero(eltype(x))   # Get a zero of the appropriate type
    @views @inbounds begin
        for i in 1:n_particles, j in i+1:n_particles
            accu += Vpair(norm(x[:, i] .- x[:, j]))
        end
    end
    accu
end

To go beyond that more global changes of the MD algorithm would be needed. For example one could store the positions and velocities of the particles as `SVector` (from `StaticArrays`), allowing the compiler to unroll the inner loops for computing the distances.

In practice one would furthermore employ a neighbour list (i.e. a list of all atoms within a certain cutoff range) to break the double loop over particles (`O(N^2)`) into a single loop over the neighbour list.

#### 2. Parallelism and speed:

Again the simplest way to parallelise the loop is using `FLoops`:

In [ ]:
using FLoops

function Vtot(Vpair, x)
    n_particles = size(x, 2)
    accu = zero(eltype(x))   # Get a zero of the appropriate type
    @floop for i in 1:n_particles
        @init local_accu = zero(eltype(x))  # Initialise thread-local accumulator
        for j in i+1:n_particles
            # Note: Since we accumulate into thread-local storage,
            # we don't need to use the blocking `@reduce` macro here.
            local_accu += @inbounds @views Vpair(norm(x[:, i] .- x[:, j]))
        end
        @reduce accu += local_accu  # Accumulate
    end
    accu
end

Again, here also more clever improvements would be possible (e.g. better chunking of the data etc.), but the coding effort will be larger.

# [02_Types_Specialisation](02_Types_Specialisation.ipynb)

1. It only needs to store the length and the position of the one-bit.

2. A possible definition of the type is:

In [ ]:
struct OneHot
    len::Int  # Length
    ind::Int  # Index of the one-bit
end 

3. To support the indicated piece of code we need the following functions frome base:

In [ ]:
import Base: *, getindex, length

length(v::OneHot) = v.len
getindex(v::OneHot, i::Int) = i == v.ind
*(A::AbstractMatrix, v::OneHot) = A[:, v.ind]

4. We benchmark the `innersum` for both indicated cases:

In [ ]:
function innersum(A, vs)
    t = zero(eltype(A))
    for v in vs
        y = A * v
        for i in 1:length(vs[1])
            t += v[i] * y[i]
        end
    end
    t
end

In [ ]:
using BenchmarkTools
A = rand(3, 3)
vs_float  = [rand(3)              for i in 1:10]
vs_onehot = [OneHot(3, rand(1:3)) for _ in 1:10]

@btime innersum($A, $vs_float);
@btime innersum($A, $vs_onehot);

In my benchmarks the speedup is about a factor of 2.

5. One way to define the `OneHotVector`:

In [ ]:
struct OneHotVector <: AbstractVector{Bool}
    len::Int
    ind::Int
end

Base.getindex(v::OneHotVector, i::Integer) = i == v.ind
Base.size(v::OneHotVector) = (v.len, )

6. Creating a single vector works ...

In [ ]:
OneHotVector(5, 3)

... and gives us a nice visualisation. Also, without any additional effort, the `innersum` just works:

In [ ]:
A  = rand(3, 3)
vs = [OneHotVector(3, rand(1:3)) for _ in 1:10]

innersum(A, vs)

# [03_Performance_Engineering](03_Performance_Engineering.ipynb)

## Optimisation project 1

In [ ]:
using BenchmarkTools

N = 100
A = rand(N, N)
b = rand(N)
c = 1.23;

#### Unoptimised code

In [ ]:
function work!(A, N)
    D = zeros(N, N)
    for i in 1:N
        D = b[i] * c * A
        b[i] = sum(D)
    end
end

@btime work!($A, $N);

First we run `@code_warntype` to check for type instabilities:

In [ ]:
@code_warntype work!(A, N)

`D` is of type `Any`, because it depends on the global variables `b` and `c`. We fix that first:

#### Avoiding globals

In [ ]:
function work2!(A, N, b, c)
    D = zeros(N, N)
    for i in 1:N
        D = b[i] * c * A
        b[i] = sum(D)
    end
end

@btime work2!($A, $N, $b, $c);

In [ ]:
@code_warntype work2!(A, N, b, c)

#### Avoiding allocations

That's fixed. Next we use vectorised operations to avoid allocations and avoid bounds checks:

In [ ]:
function work3!(A, N, b, c)
    D = zeros(N, N)
    @inbounds for i in 1:N
        @. D = b[i] * c * A
        b[i] = sum(D)
    end
end

@btime work3!($A, $N, $b, $c);

#### Improving the algorithm

The multiplication by `b[i]` and `c` can be factored out:

In [ ]:
function work4!(A, N, b, c)
    b .*= c * sum(A)
end

@btime work4!($A, $N, $b, $c);

## Optimisation project 2

In [ ]:
using BenchmarkTools

N = 4000
A = zeros(N,N)
B = rand(N,N)
v = rand(Int, N);

#### Unoptimized code

In [ ]:
function work!(A, B, v, N)
    val = 0
    for i in 1:N
        for j in 1:N
            val = mod(v[i], 256);
            A[i, j] = B[i, j] * (sin(val) * sin(val) - cos(val) * cos(val));
        end
    end
end

runtime = @belapsed work!($A, $B, $v, $N);
println("Performance: $(N^2 * 1e-6 / runtime) MIt/s")

#### Simplification

Notice:
$$
    \sin(x) \sin(x) - \cos(x) \cos(x) = 1 - 2 \cos(x) \cos(x) = - \cos(2x)
$$

In [ ]:
x = rand()
sin(x)*sin(x) - cos(x)*cos(x) ≈ -cos(2x)

In [ ]:
function work2!(A, B, v, N)
    for i in 1:N
        for j in 1:N 
            val = -cos(2mod(v[i], 256))
            A[i, j] = B[i, j] * val
        end
    end
end

runtime = @belapsed work2!($A, $B, $v, $N);
println("Performance: $(N^2 * 1e-6 / runtime) MIt/s")

#### Avoiding recomputation in the inner loop

We move the computation of the the second factor out of the inner loop:

In [ ]:
function work3!(A, B, v, N)
    for i in 1:N
        val = -cos(2mod(v[i], 256))
        for j in 1:N 
            A[i, j] = B[i, j] * val
        end
    end
end

runtime = @belapsed work3!($A, $B, $v, $N);
println("Performance: $(N^2 * 1e-6 / runtime) MIt/s")

#### Precompute val factor

In [ ]:
function work4!(A, B, v, N)
    val = -cos.(2mod.(v, 256))
    for i in 1:N
        for j in 1:N 
            A[i, j] = B[i, j] * val[i]
        end
    end
end
runtime = @belapsed work4!($A, $B, $v, $N);
println("Performance: $(N^2 * 1e-6 / runtime) MIt/s")

#### Switch loop order

In [ ]:
function work5!(A, B, v, N)
    val = -cos.(2mod.(v, 256))
    for j in 1:N
        for i in 1:N 
            A[i, j] = B[i, j] * val[i]
        end
    end
end
runtime = @belapsed work5!($A, $B, $v, $N);
println("Performance: $(N^2 * 1e-6 / runtime) MIt/s")

#### Inbounds

In [ ]:
function work6!(A, B, v, N)
    val = -cos.(2mod.(v, 256))
    @inbounds for j in 1:N
        for i in 1:N 
            A[i, j] = B[i, j] * val[i]
        end
    end
end
runtime = @belapsed work6!($A, $B, $v, $N);
println("Performance: $(N^2 * 1e-6 / runtime) MIt/s")